In [ ]:
import sys
import os

# Agrega la ruta del directorio padre (donde está src/)
sys.path.append(os.path.abspath(".."))

# 🏥 Proyecto: Predicción de Readmisión Hospitalaria
Este notebook está diseñado para ejecutarse en **Google Colab**.

Contiene todo el flujo del proyecto:
- Exploración de datos (EDA)
- Entrenamiento de modelo base (Regresión Logística)
- Entrenamiento de Red Neuronal
- Evaluación y comparación de modelos

🔧 Los estudiantes pueden modificar el código y los archivos como `config.yaml` o `train.py`.

In [ ]:
# Clonar el repositorio (solo una vez por sesión de Colab)
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git
%cd DeepNeuralNetworkUSS/Projects/HospitalReadmission


In [ ]:
# Instalar dependencias necesarias (si no están ya instaladas)
!pip install -r requirements.txt

In [ ]:
# Creacion del directorio de trabajo
%mkdir run
%cd run/

In [ ]:
# Preprocesamiento del dataset original
!python ../src/preprocess.py

In [ ]:
# Exploración de Datos (EDA)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/hospital_readmission_clean.csv')
df.head()

In [ ]:
# Modelo base: Regresión Logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sys.path.append(os.path.abspath(".."))
import src.evaluate as ev
import numpy as np
import yaml

# Cargar configuración y datos
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)
df = pd.read_csv('../data/hospital_readmission_clean.csv')
X = df.drop('readmitted', axis=1)
y = df['readmitted']

# Escalar y separar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=config['test_size'],
    random_state=config['random_state'], stratify=y)

# Entrenar modelo base
base_model = LogisticRegression()
base_model.fit(X_train, y_train)
y_pred_base = base_model.predict(X_test)
y_pred_proba_base = base_model.predict_proba(X_test)[:, 1]

# Evaluar modelo base
ev.evaluate_model(y_test, y_pred_base, y_pred_proba_base, model_name="Regresión Logística")

In [ ]:
# Entrenamiento de Red Neuronal
sys.path.append(os.path.abspath(".."))
import src.train as tr
model = tr.build_model(config, input_dim=X_train.shape[1])
history = tr.train_model(model, X_train, y_train, config)
y_pred_nn_proba = model.predict(X_test).flatten()
y_pred_nn = ev.predict_with_threshold(y_pred_nn_proba)

ev.evaluate_model(y_test, y_pred_nn, y_pred_nn_proba, model_name="Red Neuronal")

In [ ]:
# Comparación de ROC y métricas
ev.plot_roc_comparison(y_test, y_pred_nn_proba, y_test, y_pred_proba_base,
                       label1='Red Neuronal', label2='Regresión Logística')

df_metrics = ev.compare_models_metrics(
    y_test, y_pred_nn, y_pred_nn_proba,
    y_test, y_pred_base, y_pred_proba_base,
    model_name_1='Red Neuronal', model_name_2='Regresión Logística')
import pandas as pd
from IPython.display import display
display(df_metrics)